## Группировки и агрегаты

### Highlights:
- Использование параметра `index_col` в методе `read_csv()`
- Повторяем использование `pipe()` (см. функцию `convert_to_datetime_and_make_mth()`)
- Создание вспомогательного датафрейма из прямого произведения списка пользователей на список месяцев
-- Список месяц генерируется с помощью функции `pd.date_range()`
-- Произведение создается с помощью метода `pd.MultiIndex.from_product()`
- Использование "именованных агрегатов" ([named aggregations](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#named-aggregation))
- Использование конструкции `apply(lambda x: pd.Series(dict()))` для создания агрегатов с условным выражением
- Ответы на вопросы

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Читаем данные

In [2]:
users = pd.read_csv('prkt_project_users.csv', index_col = 'user_id')
calls = pd.read_csv('prkt_project_calls.csv', index_col = 'id')
messages = pd.read_csv('prkt_project_messages.csv', index_col = 'id')
sessions = pd.read_csv('prkt_project_internet.csv', index_col = 'id')

Индексная колонка крайне упрощает доступ к данным, когда в таблице есть уникальный идентификатор.

Вместо того, чтобы делать вот так:

```python
users[users['user_id'] = 1005]
```

можно делать

```python
users.loc[1005]
```

Сделаем функцию для отображения данных и информации о фрейме

In [3]:
def show_info(df):
    """Функция для отображения первых пяти строк датафрейма и информации о датафрейме"""
    display(df.head(5))
    display(df.info())

Выведем информацию по каждому фрейму

In [4]:
for df in [users,calls,messages,sessions]:
    show_info(df)

,age,churn_date,city,first_name,last_name,reg_date,tariff
user_id,,,,,,,
1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra
1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra


<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 1000 to 1499
Data columns (total 7 columns):
age           500 non-null int64
churn_date    38 non-null object
city          500 non-null object
first_name    500 non-null object
last_name     500 non-null object
reg_date      500 non-null object
tariff        500 non-null object
dtypes: int64(1), object(6)
memory usage: 31.2+ KB


None

,call_date,duration,user_id
id,,,
1000_0,2018-07-25,0.000,1000
1000_1,2018-08-17,0.000,1000
1000_2,2018-06-11,2.850,1000
1000_3,2018-09-21,13.800,1000
1000_4,2018-12-15,5.180,1000


<class 'pandas.core.frame.DataFrame'>
Index: 202607 entries, 1000_0 to 1499_219
Data columns (total 3 columns):
call_date    202607 non-null object
duration     202607 non-null float64
user_id      202607 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.2+ MB


...

<class 'pandas.core.frame.DataFrame'>
Index: 123036 entries, 1000_0 to 1499_183
Data columns (total 2 columns):
message_date    123036 non-null object
user_id         123036 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.8+ MB


None

,mb_used,session_date,user_id
id,,,
1000_0,112.950,2018-11-25,1000
1000_1,1052.810,2018-09-07,1000
1000_2,1197.260,2018-06-25,1000
1000_3,550.270,2018-08-22,1000
1000_4,302.560,2018-09-24,1000


<class 'pandas.core.frame.DataFrame'>
Index: 149396 entries, 1000_0 to 1499_156
Data columns (total 3 columns):
mb_used         149396 non-null float64
session_date    149396 non-null object
user_id         149396 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.6+ MB


None

Во фрейме `users` надо преобразовать колонки из `object` в `datetime`. Сделаем это через метод `assign()` (см. консультацию "Method Chaining"):

In [5]:
users = users.assign(
    churn_date = lambda x: pd.to_datetime(x['churn_date']),
    reg_date = lambda x: pd.to_datetime(x['reg_date'])
)

Для остальных фреймов сделаем функцию, которая позволяет преобразовать колонку с датой в `datetime` и дополнительно создать колонку с месяцем. Обратите внимание, что для преобразования в месяц используется метод `astype()` с параметром `datetime64[M]`. Это позволяет избежать возможных ошибок при использовании ацессора `dt.month` (например, если данные за несколько лет).

In [6]:
def convert_to_datetime_and_make_mth(df,column_name,month_column_name):
    """Функция для преобразования колонки column_name датафрейма df
    в дату и создания новой колонки month_column_name с месяцем на колонки column_name"""
    df = df.copy()
    df[column_name] = pd.to_datetime(df[column_name])
    df[month_column_name] = df[column_name].astype('datetime64[M]')
    return df

Применим функцию с помощью метода `pipe()` (см. консультацию "Method Chaining"):

In [7]:
calls = calls.pipe(convert_to_datetime_and_make_mth,'call_date','month')
messages = messages.pipe(convert_to_datetime_and_make_mth,'message_date','month')
sessions = sessions.pipe(convert_to_datetime_and_make_mth,'session_date','month')

Создадим вспомогательный фрейм, который будет содержать прямое произведение (cartesian product) всех месяцев и всех пользователей

In [8]:
calls['month'].sort_values().unique()

array(['2018-01-01T00:00:00.000000000', '2018-02-01T00:00:00.000000000',
       '2018-03-01T00:00:00.000000000', '2018-04-01T00:00:00.000000000',
       '2018-05-01T00:00:00.000000000', '2018-06-01T00:00:00.000000000',
       '2018-07-01T00:00:00.000000000', '2018-08-01T00:00:00.000000000',
       '2018-09-01T00:00:00.000000000', '2018-10-01T00:00:00.000000000',
       '2018-11-01T00:00:00.000000000', '2018-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

Задаем последовательность месяцев с помощью функции `pd.date_range()`

In [9]:
month_range = pd.date_range(start=calls['month'].min(),end=calls['month'].max(),freq='MS')

In [10]:
user_range = users.index

Делаем фрейм с прямым произведением в индексе

In [11]:
user_mth_index = pd.DataFrame(index=pd.MultiIndex.from_product([user_range,month_range])).reset_index()

In [12]:
user_mth_index.columns = ['user_id','month']

Перейдем к группировке данных по пользователям и месяцам.

**Первый способ:** Возвращает Series. Поэтому если хотим получить фрейм, то нужно делать `to_frame()`, а затем переименовать колонку, чтобы сделать правильное обозначение содержимого

In [13]:
messages.groupby(['user_id','month'])['message_date'].count().\
to_frame().reset_index().\
rename(columns={'message_date':'messages'}).\
head()

,user_id,month,messages
0,1000,2018-05-01,22
1,1000,2018-06-01,60
2,1000,2018-07-01,75
3,1000,2018-08-01,81
4,1000,2018-09-01,57


**Второй способ:** использовать метод `agg()`. Тут уже не нужно делать `to_frame()`, т.к. возвращается датафрейм, но при этом всё равно надо переименовывать. Преимущество `agg()` в том, что можно задать в рамках одной операции несколько колонок для агрегации, а также несколько агрегирующих функций для одной колонки.

In [14]:
messages.groupby(['user_id','month']).agg({'message_date':'count'}).reset_index().\
rename(columns={'message_date':'messages'}).\
head()

,user_id,month,messages
0,1000,2018-05-01,22
1,1000,2018-06-01,60
2,1000,2018-07-01,75
3,1000,2018-08-01,81
4,1000,2018-09-01,57


**Третий способ**: использовать named aggregations (https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#named-aggregation). Такой способ агреграции позволяет сразу же задать название колонки, которая будет на выходе после процесса агрегации

In [15]:
messages_per_month = messages.groupby(['user_id','month']).agg(messages = ('message_date','count')).reset_index()

In [16]:
sessions_per_month = sessions.groupby(['user_id','month']).agg(mb_used = ('mb_used','sum')).reset_index()

В наших данных каждая строка обозначает отдельный звонок, таким образом на каждую дату для каждого пользователя может приходится несколько звонков. При этом часть из них будет с нулевой длительностью. Посчитаем таблицу со сгруппированными по месяцам данными, но при этом посчитав отдельно количество звонков с ненулевой длительностью

In [17]:
calls.groupby(['user_id','call_date']).agg({'duration':'count'}).sort_values('duration',ascending=False).head()

,,duration
user_id,call_date,
1336,2018-12-31,100
1140,2018-12-31,75
1258,2018-12-31,39
1485,2018-12-29,38
1074,2018-12-31,38


Для того, чтобы посчитать количество звонков с ненулевой длительностью, используем хитрую конструкцию, позволяющую посчитать агрегат не по всем столбцу, а применив перед этим определенное условие (длительность > 0):

In [18]:
calls_per_month = calls.groupby(['user_id','month']).apply(lambda x: pd.Series(dict(
    calls_duration = x['duration'].sum(),
    calls_count_with_duration = x[x['duration'] > 0]['duration'].count(),
    calls_count_total = x['duration'].count()))).reset_index().\
assign(calls_with_duration_share = lambda x: x['calls_count_with_duration'] / x['calls_count_total']) # И сразу считаем долю


In [19]:
print(calls_per_month['calls_with_duration_share'].median())

0.8055555555555556


In [20]:
print(calls_per_month['calls_with_duration_share'].mean())

0.8034997118337494


Сделаем таблицу с объединением всех таблиц. За основу возьмем вспомогательную таблицу

In [21]:
user_behavior = user_mth_index.\
merge(calls_per_month,on=['user_id','month'],how='left').\
merge(messages_per_month,on=['user_id','month'],how='left').\
merge(sessions_per_month,on=['user_id','month'],how='left').\
merge(users,how='left',left_on='user_id',right_on='user_id')

После объединения получили лишние строки (из-за вспомогательной таблицы). Причины их возникновения:
- Если месяц меньше месяца регистрации
- Если месяц больше месяца ухода

In [22]:
user_behavior[['user_id','month','reg_date']].head(10).\
assign(delta = lambda x: (x['reg_date'] - x['month']).dt.days)

,user_id,month,reg_date,delta
0,1000,2018-01-01,2018-05-25,144
1,1000,2018-02-01,2018-05-25,113
2,1000,2018-03-01,2018-05-25,85
3,1000,2018-04-01,2018-05-25,54
4,1000,2018-05-01,2018-05-25,24
5,1000,2018-06-01,2018-05-25,-7
6,1000,2018-07-01,2018-05-25,-37
7,1000,2018-08-01,2018-05-25,-68
8,1000,2018-09-01,2018-05-25,-99
9,1000,2018-10-01,2018-05-25,-129


Избавимся от этих лишних строк

In [23]:
user_behavior['reg_month'] = user_behavior['reg_date'].astype('datetime64[M]')
user_behavior['churn_month'] = user_behavior['churn_date'].astype('datetime64[M]')
user_behavior = user_behavior[(user_behavior['month'] >= user_behavior['reg_month'])]
user_behavior = user_behavior[(user_behavior['month'] <= user_behavior['churn_month']) | (user_behavior['churn_date'].isnull())]

Есть два пользователя, у которых не было никаких данных. Если бы мы объединяли без вспомогательной таблицы, то такие пользователи не попали бы в объединенную таблицу. Теперь мы знаем, что у нас было два пользователя, которые не пользовались услугами в месяце подключения. При этом они уходят в тот же месяц, что и подключились.

Если бы у определенных пользователей были "пробелы" в использовании услуг, например, один из месяцев они были бы неактивны (не использовали никакие услуги), то такая объединенная таблица также позволила бы их найти. Но в наших данных таких пользователей нет.

In [24]:
user_behavior[(user_behavior['calls_count_with_duration'].isnull())
              & (user_behavior['messages'].isnull())
              & (user_behavior['mb_used'].isnull())]

,user_id,month,calls_duration,calls_count_with_duration,calls_count_total,calls_with_duration_share,messages,mb_used,age,churn_date,city,first_name,last_name,reg_date,tariff,reg_month,churn_month
1547,1128,2018-12-01,nan,nan,nan,nan,nan,nan,51,2018-12-23,Волжский,Ксения,Агаева,2018-12-15,ultra,2018-12-01,2018-12-01
4463,1371,2018-12-01,nan,nan,nan,nan,nan,nan,50,2018-12-30,Омск,Ириней,Трофимов,2018-12-25,smart,2018-12-01,2018-12-01


## Ответы на вопросы

Как получить название месяца

In [25]:
calls_per_month['month'].dt.month_name()

0             May
1            June
2            July
3          August
4       September
          ...    
3169      October
3170    September
3171      October
3172     November
3173     December
Name: month, Length: 3174, dtype: object

Как получить месяц + год

In [26]:
calls_per_month['month'].apply(lambda x: x.month_name() + ' ' + str(x.year))

0             May 2018
1            June 2018
2            July 2018
3          August 2018
4       September 2018
             ...      
3169      October 2018
3170    September 2018
3171      October 2018
3172     November 2018
3173     December 2018
Name: month, Length: 3174, dtype: object

Перенос строки в `query()`

In [27]:
calls_per_month.\
query('calls_duration > 0 & \
       calls_count_total > 0').\
head()

,user_id,month,calls_duration,calls_count_with_duration,calls_count_total,calls_with_duration_share
0,1000,2018-05-01,150.060,17.000,22.000,0.773
1,1000,2018-06-01,158.900,28.000,43.000,0.651
2,1000,2018-07-01,318.800,41.000,47.000,0.872
3,1000,2018-08-01,389.870,42.000,52.000,0.808
4,1000,2018-09-01,440.140,46.000,58.000,0.793


Получить день через `astype()`

In [28]:
calls_per_month['month'].astype('datetime64[D]')

0      2018-05-01
1      2018-06-01
2      2018-07-01
3      2018-08-01
4      2018-09-01
          ...    
3169   2018-10-01
3170   2018-09-01
3171   2018-10-01
3172   2018-11-01
3173   2018-12-01
Name: month, Length: 3174, dtype: datetime64[ns]

Обратите внимание на разницу типов. Ацессор `dt.date` выдаёт `object`. А `astype('datetime64[D]')` возвращает `datetime64[ns]`

In [29]:
calls_per_month['month'].dt.date

0       2018-05-01
1       2018-06-01
2       2018-07-01
3       2018-08-01
4       2018-09-01
           ...    
3169    2018-10-01
3170    2018-09-01
3171    2018-10-01
3172    2018-11-01
3173    2018-12-01
Name: month, Length: 3174, dtype: object

Как получить год?

In [30]:
calls_per_month['month'].dt.year

0       2018
1       2018
2       2018
3       2018
4       2018
        ... 
3169    2018
3170    2018
3171    2018
3172    2018
3173    2018
Name: month, Length: 3174, dtype: int64

Либо через `astype()`:

In [31]:
calls_per_month['month'].astype('datetime64[Y]')

0      2018-01-01
1      2018-01-01
2      2018-01-01
3      2018-01-01
4      2018-01-01
          ...    
3169   2018-01-01
3170   2018-01-01
3171   2018-01-01
3172   2018-01-01
3173   2018-01-01
Name: month, Length: 3174, dtype: datetime64[ns]